In [61]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [62]:
true_path = "C:\\Users\\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_stbd.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
prediction_path = "C:\\Users\\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\DL_PredictedCSVs\\DL_PredictedOutputTest_Row33_3_15_2022.csv"

In [63]:
df_true=pd.read_csv(true_path)
#df_test=pd.read_csv(test_path)
df_prediction=pd.read_csv(prediction_path)


In [64]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["PredictedDOA"]]
df_predictedDOA

,PredictedDOA
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
5224712,81.370316
5224713,81.358650
5224714,81.354164
5224715,81.352692


In [65]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference

,DOA,AngleUncertainty
0,72.318001,0
1,71.376001,0
2,70.536001,0
3,69.780001,0
4,69.090001,0
...,...,...
5224712,80.334001,28
5224713,79.836001,30
5224714,79.698001,35
5224715,79.878001,31


In [66]:
df_reference['AngleUncertainty'].max()

255

In [67]:
df_reference['AngleUncertainty'].mean()

57.80040239500053

In [68]:
df_reference['AngleUncertainty'].var()

4524.441105398652

In [69]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

,PredictedDOA,DOA,AngleUncertainty
0,0.000000,72.318001,0
1,0.000000,71.376001,0
2,0.000000,70.536001,0
3,0.000000,69.780001,0
4,0.000000,69.090001,0
...,...,...,...
5224712,81.370316,80.334001,28
5224713,81.358650,79.836001,30
5224714,81.354164,79.698001,35
5224715,81.352692,79.878001,31


In [70]:
theShape = df_comparison.shape
numRows= theShape[0]

In [71]:
numCols = 1
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail"}
df_results

,PassFail
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5224712,NaN
5224713,NaN
5224714,NaN
5224715,NaN


In [72]:
df_comparison = df_comparison.join(df_results)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail
0,0.000000,72.318001,0,NaN
1,0.000000,71.376001,0,NaN
2,0.000000,70.536001,0,NaN
3,0.000000,69.780001,0,NaN
4,0.000000,69.090001,0,NaN
...,...,...,...,...
5224712,81.370316,80.334001,28,NaN
5224713,81.358650,79.836001,30,NaN
5224714,81.354164,79.698001,35,NaN
5224715,81.352692,79.878001,31,NaN


In [73]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,72.318001,0,NaN,-72.318001,72.318001
1,0.000000,71.376001,0,NaN,-71.376001,71.376001
2,0.000000,70.536001,0,NaN,-70.536001,70.536001
3,0.000000,69.780001,0,NaN,-69.780001,69.780001
4,0.000000,69.090001,0,NaN,-69.090001,69.090001
...,...,...,...,...,...,...
5224712,81.370316,80.334001,28,NaN,1.036315,1.036315
5224713,81.358650,79.836001,30,NaN,1.522650,1.522650
5224714,81.354164,79.698001,35,NaN,1.656163,1.656163
5224715,81.352692,79.878001,31,NaN,1.474691,1.474691


In [74]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,72.318001,0,False,-72.318001,72.318001
1,0.000000,71.376001,0,False,-71.376001,71.376001
2,0.000000,70.536001,0,False,-70.536001,70.536001
3,0.000000,69.780001,0,False,-69.780001,69.780001
4,0.000000,69.090001,0,False,-69.090001,69.090001
...,...,...,...,...,...,...
5224712,81.370316,80.334001,28,False,1.036315,1.036315
5224713,81.358650,79.836001,30,False,1.522650,1.522650
5224714,81.354164,79.698001,35,False,1.656163,1.656163
5224715,81.352692,79.878001,31,False,1.474691,1.474691


In [75]:
PassCount = df_comparison['PassFail'].sum()
PassCount

607233

In [76]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount

4617484

In [77]:
TotalCount = PassCount+FailCount
TotalCount

5224717

In [78]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass

11.62231370617777